In [31]:
# importing libraries
from constants import *
import os
from dotenv import load_dotenv
import subprocess
import json
from eth_account import Account
from bit import PrivateKeyTestnet
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from bit.network import NetworkAPI

load_dotenv()

True

In [32]:
# calling mnemonic environment variable
mnemonic = os.getenv('MNEMONIC')

In [33]:
# function to derive wallets
def derive_wallets (mnemonic, coin, number):
    
    command = f"./derive -g --mnemonic='{mnemonic}' --coin='{coin}' --numderive='{number}' --cols=address,index,path,address,privkey,pubkey,pubkeyhash,xprv,xpub --format=json"
    process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = process.communicate()
    process_status = process.wait()
    keys = json.loads(output)
    
    return keys

In [34]:
# function to call derive_wallets function for BTCTEST and ETH currencies and output a dictionary
# with derived information for three accounts per coin
def coins ():

    coin_dict = {
        'btc-test' : derive_wallets(mnemonic, BTCTEST, 3),
        'eth' : derive_wallets(mnemonic, ETH, 3)
    }
    
    return coin_dict

In [35]:
# testing output
coins()

{'btc-test': [{'address': 'mypsQPyrYT856zfsAaibg2r95sS4TDbV2c',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cV1dmp5Lyi4X4azbVGQykema2CbeF3PJaV1R75XyBo1hyszhNhtB',
   'pubkey': '02bd5769a9f1296e368e14a0d5869b1636266c2bdccc9ee4469ff17b839bb3993b',
   'pubkeyhash': 'c8d5ab3e6df746323c120563a6c5b6b17591755b',
   'xprv': 'tprv8jRjQPYU6MJ93ADJQBQwmigVk3c1Qgp7cpNjJsTfMVoQLQgX65KfzV17LU6TvdvVtC84FjpBWPuuEzCLjitNBeoDRsGpJnf9dqHZt4LKypx',
   'xpub': 'tpubDG7mYoaiEiyovdF6Hq5YB8LcK57wa212C7yWbPVxmmboAtwHiU9GAycyWao7dSVzpxDijvrFW3UgsP656Vy7R4fZ2dmE6JGfznSRbx98rjZ'},
  {'address': 'n29MHzRVyDF1DQMHoCeD1PTWf6Voo9JqdN',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cQAiD4iXaf9xy7cFZQY2ehbJZMQJoAAdJEavqHv1rxSaSZDP28os',
   'pubkey': '0347cce01b0152ae95d15253f2e01945219730ae5af67e06a926545962209da553',
   'pubkeyhash': 'e244b056c5739cde7e407749fd0201e67e55d2ce',
   'xprv': 'tprv8jRjQPYU6MJ96E8nZVG1RhSSdC2jHsLS5nhwV65YZv9PJVYyWarjqbfZosGxzg4iFQv6qYsbR852n1EEjiT9YaKbQRNdW8zPU

In [36]:
# testing a call of a private key from the dictionary output
coins()[ETH][0]['privkey']

'0x35e62d4046ed76da7d209969e10563895a43b7d6a3cd9802e53802ff03948da3'

In [37]:
# function to convert private key into a readable format for web3 / bit
def priv_key_to_account (coin, priv_key):
    
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [38]:
# setting sender ETH account
eth_account = priv_key_to_account(ETH,coins()[ETH][0]['privkey'])

print(eth_account)

In [39]:
# setting a receiver ETH address
eth_address = coins()[ETH][1]['address']

print(eth_address)

0x501e9dc485842A11a83B2FEc16ec8C6079DCdc2C


In [40]:
# setting sender BTCTEST account
btctest_account = priv_key_to_account(BTCTEST,coins()[BTCTEST][0]['privkey'])

print(btctest_account)

<PrivateKeyTestnet: mypsQPyrYT856zfsAaibg2r95sS4TDbV2c>


In [41]:
# setting a receiver BTCTEST address
btctest_address = coins()[BTCTEST][1]['address']

print(btctest_address)

n29MHzRVyDF1DQMHoCeD1PTWf6Voo9JqdN


In [42]:
# setting up Web3 port
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [43]:
# function to create raw, unsigned transaction
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({
            "from": account.address, 
            "to": to, 
            "value": w3.toWei(amount,'ether') 
        })
        return {
            "from": account.address,
            "to": to,
            "value": w3.toWei(amount,'ether') ,
            "gas": gasEstimate,
            "gasPrice": w3.eth.gasPrice,
            "nonce": w3.eth.getTransactionCount(account.address),
            #"chainID": w3.net.chainId --this has been deprecated
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [44]:
# testing transaction with ETH
create_tx(ETH,eth_account,eth_address,1)

{'from': '0x51e4767C3c9075C515140FeE00Fa82d4749C3a25',
 'to': '0x501e9dc485842A11a83B2FEc16ec8C6079DCdc2C',
 'value': 1000000000000000000,
 'gas': 21000,
 'gasPrice': 20000000000,
 'nonce': 5}

In [45]:
# testing transaction with BTC
create_tx(BTCTEST,btctest_account,btctest_address,0.001)

'{"unspents":[{"amount":1570132,"confirmations":4,"script":"76a914c8d5ab3e6df746323c120563a6c5b6b17591755b88ac","txid":"7a0a6dd353fcdb212ae3ee1cd1e024282c39cc09c7c8c242da274d7e5f022ac4","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["n29MHzRVyDF1DQMHoCeD1PTWf6Voo9JqdN",100000],["mypsQPyrYT856zfsAaibg2r95sS4TDbV2c",1442560]]}'

In [46]:
# function to create, sign, and send transaction 
def send_tx(coin, account, recipient, amount):
    
    raw_tx = create_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(raw_tx)
    
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [47]:
# sending ETH transaction
send_tx(ETH,eth_account,eth_address,1)

HexBytes('0x1b1c0f8c2b973c298a31947fe2d8d52f6095530572b550f4b5e0512b1081aed2')

In [48]:
# sending BTCTEST transaction
send_tx(BTCTEST,btctest_account,btctest_address,0.001)